In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve,auc,accuracy_score

In [ ]:
social=pd.read_csv("/kaggle/input/social-network-ads/Social_Network_Ads.csv")

In [ ]:
social.describe()

In [ ]:
social.columns

In [ ]:
social.dtypes

In [ ]:
social.isnull().sum()

In [ ]:
social.duplicated().sum()
social.drop_duplicates(inplace=True)

In [ ]:
x=social.drop("Purchased",axis=1, inplace=False)
x=x.values
y=social["Purchased"].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

LR=LogisticRegression(solver="lbfgs",class_weight="balanced",max_iter=10000)
model_LR=LR.fit(x_train,y_train)
KNN=KNeighborsClassifier()
model_KNN=KNN.fit(x_train,y_train)
DTC=DecisionTreeClassifier(random_state=0)
model_DTC=DTC.fit(x_train,y_train)
RFC=RandomForestClassifier(random_state=0)
model_RFC=RFC.fit(x_train,y_train)
GBC=GradientBoostingClassifier(random_state=0)
model_GBC=GBC.fit(x_train,y_train)

In [ ]:
models=[model_LR,model_KNN,model_DTC,model_RFC,model_GBC]

for model in models:
    name=model.__class__.__name__
    R2=cross_val_score(model,x_test,y_test,cv=10,verbose=False).mean()
    error=-cross_val_score(model,x_test,y_test,cv=10,scoring="neg_mean_squared_error",verbose=False).mean()
    print(name + ":")
    print("*"*20)
    print(R2)
    print(np.sqrt(error))
    print("*"*20)

# Random Forest Classifier is giving the best result.

In [ ]:
prediction=RFC.predict(x_test)
print(confusion_matrix(y_test,prediction))

In [ ]:
print(classification_report(y_test,prediction))

In [ ]:
P=RFC.predict_proba(x_test)[:,1]
fpr,tpr,threshold=roc_curve(y_test,P)
plt.figure()
plt.plot(fpr,tpr)
plt.plot([0,1],[0,1],"r--")
plt.xlim(0,1)
plt.ylim(0,1.05)
plt.show()

In [ ]:
area=auc(fpr,tpr)
print(area.round(2))

In [ ]:
accuracy_score(y_test,prediction)